In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings

warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
tianjin = '117.4961975657171,37.86803012379458,7.296245301137998 119.8081837995413,37.9250916349774,-34.2681049860788 119.8416643486643,39.32481316399956,-31.12524274095152 117.9501096884913,39.37523731857854,39.41212136434525 117.4961975657171,37.86803012379458,7.296245301137998'



coord_list = tianjin.split(" ")
coord_list
coord_list2 = []
for i in coord_list:
    temp = i.split(",")[:2]
    temp = list(map(float, temp))
    coord_list2.append(temp)
poly = Polygon(coord_list2)
# if Point(LONGITUDE, LATITUDE).within(poly):

In [3]:
df_test = pd.read_csv("voy_path/COSCO_FOS_VRKR6_25.csv")
df_test

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.5569,34.0799,18.4,280.0,2022-11-08 18:56:13.959541,2022-11-08 09:55:53,2022-11-08 07:31:08,under way using engine,229.666166,452.388243
1,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.5261,34.0849,18.3,285.0,2022-11-08 19:01:17.137621,2022-11-08 10:01:01,2022-11-08 07:31:08,under way using engine,231.973583,453.620310
2,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4980,34.0905,18.5,280.0,2022-11-08 19:06:19.944782,2022-11-08 10:05:41,2022-11-08 07:31:08,under way using engine,234.039994,464.373403
3,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4644,34.0925,18.8,275.0,2022-11-08 19:11:23.235915,2022-11-08 10:11:06,2022-11-08 07:31:08,under way using engine,236.745812,480.132626
4,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4325,34.0943,18.9,275.0,2022-11-08 19:16:26.088005,2022-11-08 10:16:10,2022-11-08 07:31:08,under way using engine,239.327692,488.925120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8127,35.0728,6.0,80.0,2022-11-14 15:45:30.478230,2022-11-14 06:42:40,2022-11-14 06:43:07,under way using engine,11.491783,17.159610
484,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8261,35.0755,1.4,76.0,2022-11-14 16:00:40.036669,2022-11-14 06:58:29,2022-11-14 06:49:08,under way using engine,11.791119,67.307567
485,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8294,35.0774,0.0,90.0,2022-11-14 16:41:02.521600,2022-11-14 07:39:30,2022-11-14 07:37:07,moored,12.021023,NaN
486,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8293,35.0774,0.1,91.0,2022-11-14 17:26:27.030687,2022-11-14 08:24:24,2022-11-14 08:25:09,moored,12.020340,959.128537


In [4]:
# if Point(LONGITUDE, LATITUDE).within(poly):

In [5]:
df_test.sort_values(by="timestamp", ignore_index=True, inplace=True)

In [6]:
for idx in df_test.index:
    LONGITUDE = df_test.loc[idx, "longitude"]
    LATITUDE = df_test.loc[idx, "latitude"]
    heading = df_test.loc[idx, "heading"]
    speed = df_test.loc[idx, "speed"]
    if Point(LONGITUDE, LATITUDE).within(poly) and heading < 200 and speed > 11:
        print(idx)
        break
        

282


In [7]:
df_test_2 = df_test.loc[idx:,]
df_test_2.reset_index(inplace=True)

In [8]:
# df_test_2.to_csv("voy_path/COSCO_test.csv", encoding="utf-8 sig", header=True, index=False)

In [9]:
distance = 0
for idx in df_test_2.index:
    if idx != 0:
        LONGITUDE = df_test_2.loc[idx, "longitude"]
        LATITUDE = df_test_2.loc[idx, "latitude"]
        current_coord = (LATITUDE, LONGITUDE)

        LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
        LATITUDE = df_test_2.loc[idx - 1, "latitude"]
        previous_current_coord = (LATITUDE, LONGITUDE)

        distance += haversine(current_coord, previous_current_coord, unit="km")

In [10]:
distance

1309.6651548009452

In [11]:
1348.5265119662952 / 20

67.42632559831476

In [12]:
overall_length = 1348.5265119662952
segment_length = overall_length/20

In [13]:
overall_length = distance
segment_length = overall_length/20

In [14]:
segment_length

65.48325774004726

In [15]:
temp_accu_length = 0
segmenting_index_list = []
for idx in df_test_2.index:
    
    if idx == 0:
        previous_length = 0
    else:
        LONGITUDE = df_test_2.loc[idx, "longitude"]
        LATITUDE = df_test_2.loc[idx, "latitude"]
        current_coord = (LATITUDE, LONGITUDE)

        LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
        LATITUDE = df_test_2.loc[idx - 1, "latitude"]
        previous_current_coord = (LATITUDE, LONGITUDE)

        current_length = haversine(current_coord, previous_current_coord, unit="km")
        temp_accu_length += current_length
        
        if temp_accu_length >= segment_length:
            print(temp_accu_length)
#             segmenting_index_list.append(idx)
            
            
            length_error_1 = temp_accu_length - segment_length
            length_error_2 = temp_accu_length - segment_length - previous_length
            if length_error_1 > length_error_2:
                segmenting_index_list.append(idx)
            else:
                segmenting_index_list.append(idx - 1)
                
            
            temp_accu_length = 0
            
        previous_length = current_length
segmenting_index_list.append(df_test_2.index[-1])


72.62727212414536
142.95396628771988
65.51016254782469
80.9702637044401
69.41110803973697
74.1397948130414
300.7040847086696
67.848224269958
69.4899241896433
69.54566403434967
67.23005539201127
66.08107794443198
66.17147192444024
65.76426522138014


In [16]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [17]:
for idx, index in enumerate(segmenting_index_list):
    if idx == 0:
        d = df_test_2[0:index + 1]
#         display(d)
        distance = reckon_point_lengths(d)
        print(distance)
        d.to_csv(f"실험/segment_test_new_{idx}.csv", encoding="utf-8 sig", header=True, index=False)
    else:
        d = df_test_2[previous_index:index + 1]
#         display(d)
        distance = reckon_point_lengths(d)
        print(distance)
        d.to_csv(f"실험/segment_test_new_{idx}.csv", encoding="utf-8 sig", header=True, index=False)
        
    previous_index = index

72.62727212414536
142.95396628771988
65.51016254782469
80.9702637044401
69.41110803973697
74.1397948130414
300.7040847086696
67.848224269958
69.4899241896433
69.54566403434967
67.23005539201127
66.08107794443198
66.17147192444024
65.76426522138014
31.21781959915293


In [18]:
for i in range(len(segmenting_index_list)):
    d = pd.read_csv(f"실험/segment_test_new_{i}.csv")
    mean_speed = d["speed"].mean()
    print(i, mean_speed)

0 15.317647058823528
1 17.52857142857143
2 17.32857142857143
3 18.83333333333333
4 18.6
5 19.466666666666665
6 19.75
7 18.95
8 18.176
9 18.757142857142856
10 19.6625
11 18.539130434782603
12 17.941666666666666
13 13.190909090909093
14 7.034999999999999


In [19]:
print("COSCO_FOS_25")
print("전항지 출항 시각",df_test_2.loc[df_test_2.index[0], "timestamp"])
print("부산 도착 시각",df_test_2.loc[df_test_2.index[-1], "timestamp"])

COSCO_FOS_25
전항지 출항 시각 2022-11-12 23:02:38.554501
부산 도착 시각 2022-11-14 18:47:12.361130


In [36]:
sample = pd.read_csv("voy_path/COSCO_FOS_VRKR6_26.csv")
sample.sort_values(by="timestamp", ignore_index=True, inplace=True)
for idx in sample.index:
    LONGITUDE = sample.loc[idx, "longitude"]
    LATITUDE = sample.loc[idx, "latitude"]
    heading = sample.loc[idx, "heading"]
    speed = sample.loc[idx, "speed"]
    if Point(LONGITUDE, LATITUDE).within(poly) and heading < 200 and speed > 11:
        print(idx)
        break
sample = sample.loc[idx:,]
sample.reset_index(inplace=True)


224


In [37]:
def cal_duration(speed, distance):
    hour = int(distance // km_h)
    minute = int(
        (distance / km_h - distance // km_h) * 60)
    duration = datetime.timedelta(hours=hour,minutes=minute)
    return duration

In [38]:
# 억까 방지
busan_berth = '128.7362432822305,34.91571835540296,5.941836244531453 128.8816505843355,34.93956159255044,9.088787041386029 128.8718190496849,35.09310779849003,8.308118991212194 128.7247817257968,35.08647052441917,-9.228016031556635 128.7362432822305,34.91571835540296,5.941836244531453'
coord_list = busan_berth.split(" ")
coord_list
coord_list2 = []
for i in coord_list:
    temp = i.split(",")[:2]
    temp = list(map(float, temp))
    coord_list2.append(temp)
poly_busan = Polygon(coord_list2)
# if Point(LONGITUDE, LATITUDE).within(poly):

In [39]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

In [40]:
sample

,index,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,224,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,117.8339,38.9635,11.5,101.0,2022-11-19 22:24:43.599696,2022-11-19 13:20:25,2022-11-19 01:07:06,under way using engine,1071.303386,1386.146465
1,225,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,117.8521,38.9608,11.6,102.0,2022-11-19 22:29:46.266674,2022-11-19 13:25:02,2022-11-19 01:07:06,under way using engine,1069.727209,1388.920303
2,226,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,117.8962,38.9541,11.5,101.0,2022-11-19 22:39:53.134121,2022-11-19 13:35:45,2022-11-19 01:31:06,under way using engine,1065.901997,1379.157672
3,227,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,117.9377,38.9484,11.6,102.0,2022-11-19 22:49:57.626655,2022-11-19 13:46:03,2022-11-19 01:49:08,under way using engine,1062.327066,1379.312051
4,228,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,117.9571,38.9454,11.6,102.0,2022-11-19 22:55:01.041397,2022-11-19 13:50:45,2022-11-19 01:49:08,under way using engine,1060.642543,1377.124888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,528,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8286,35.0767,0.9,80.0,2022-11-21 18:05:52.289081,2022-11-21 08:45:01,2022-11-21 08:37:08,under way using engine,11.938104,105.884565
305,529,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8303,35.0770,0.1,91.0,2022-11-21 20:12:03.039219,2022-11-21 10:46:03,2022-11-21 10:49:07,moored,11.983151,956.161110
306,530,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8302,35.0772,0.0,91.0,2022-11-21 20:17:05.890117,2022-11-21 10:52:05,2022-11-21 10:55:06,moored,12.004568,NaN
307,531,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8300,35.0772,0.0,91.0,2022-11-21 20:42:20.130436,2022-11-21 11:15:59,2022-11-21 11:13:06,moored,12.003097,NaN


In [41]:
stop_index = check_ATA(sample)

In [42]:
stop_index

295

In [43]:
sample.loc[stop_index:]

,index,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
295,519,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8326,34.9308,9.7,345.0,2022-11-21 16:50:10.106911,2022-11-21 07:29:14,2022-11-21 07:25:09,under way using engine,4.477744,5.594956
296,520,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8316,34.9334,9.7,345.0,2022-11-21 16:55:12.370789,2022-11-21 07:30:20,2022-11-21 07:31:07,under way using engine,4.174799,5.216425
297,521,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8109,34.9799,11.9,343.0,2022-11-21 17:10:24.772941,2022-11-21 07:46:50,2022-11-21 07:49:08,under way using engine,1.385139,1.818734
298,522,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.7941,35.0195,12.4,341.0,2022-11-21 17:20:26.867541,2022-11-21 08:00:05,2022-11-21 07:49:08,under way using engine,6.009961,8.060099
299,523,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.7889,35.0318,10.6,341.0,2022-11-21 17:25:29.125227,2022-11-21 08:04:23,2022-11-21 07:49:08,under way using engine,7.455118,9.412492
300,524,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.7868,35.0597,8.2,25.0,2022-11-21 17:35:35.996969,2022-11-21 08:15:14,2022-11-21 08:13:09,under way using engine,10.450602,13.357489
301,525,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.7908,35.0657,8.0,53.0,2022-11-21 17:40:38.692701,2022-11-21 08:18:19,2022-11-21 08:13:09,under way using engine,10.999747,14.165268
302,526,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8049,35.0702,7.1,75.0,2022-11-21 17:45:41.601632,2022-11-21 08:24:32,2022-11-21 08:13:09,under way using engine,11.264421,15.235716
303,527,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8218,35.0740,3.9,69.0,2022-11-21 17:55:45.279861,2022-11-21 08:34:22,2022-11-21 08:13:09,under way using engine,11.610739,24.556266
304,528,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8286,35.0767,0.9,80.0,2022-11-21 18:05:52.289081,2022-11-21 08:45:01,2022-11-21 08:37:08,under way using engine,11.938104,105.884565


In [44]:
ATA = pd.to_datetime(sample.loc[stop_index,"timestamp"])
ATA

Timestamp('2022-11-21 16:50:10.106911')

# 감속률 적용

In [45]:
len(segmenting_index_list)

15

In [46]:
sam

306

In [31]:
# 샘플 데이터는 20부터. 
for sam in sample.index[20:stop_index]:
    re_sample = sample.loc[sam,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    current_speed = re_sample["speed"]
    current_timestamp = re_sample["timestamp"]
    current_timestamp = pd.to_datetime(current_timestamp)
    
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 999
    for i in range(len(segmenting_index_list)):
        d = pd.read_csv(f"실험/segment_test_new_25_{i}.csv")
        for idx in d.index:

            LONGITUDE = d.loc[idx, "longitude"]
            LATITUDE = d.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance < shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = d.loc[idx, "index"]
                seg_no = i
#     print(shortest_distance)
    # 마지막 인덱스 제외
    if shortest_point_idx == df_test_2.loc[df_test_2.index[-1],"index"]:
        continue
        
    # 가장 가까운 점에서 다음 점 (n+1) 선택. 
    # n+1 점과 현재 점
    df__ = df_test_2.query(f'index == {shortest_point_idx + 1}')
    new_point = (df__["latitude"].values[0],df__["longitude"].values[0])
    distance = haversine(current_point, new_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, distance)
    
    # 가장 가까운 segment 데이터 불러오기
    init_seg_df = pd.read_csv(f"실험/segment_test_new_{seg_no}.csv")
    # 마지막 세그먼트라면 제외
    if len(init_seg_df.query(f'index == {shortest_point_idx + 1}')) == 0:
        init_seg_df = pd.read_csv(f"실험/segment_test_new_{seg_no + 1}.csv")
        starting_index = init_seg_df.index[0]
    else:
        starting_index = init_seg_df.query(f'index == {shortest_point_idx + 1}').index[0]
        
    # 감속률 계산
    speed_damping_ratio = current_speed / init_seg_df["speed"].mean()
    
    # 남은 segment 간의 거리 계산 
    remained_init_seg_df = init_seg_df.loc[starting_index:,]
    remained_init_seg_df.reset_index(drop=True, inplace=True)
    accu_distance = 0
    for idx in remained_init_seg_df.index:
        if idx != 0:
            LONGITUDE = remained_init_seg_df.loc[idx, "longitude"]
            LATITUDE = remained_init_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remained_init_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remained_init_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
    # 첫번쨰 segment duration 계산
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    # 나머지 segment 계산
    # 억까 방지를 위해 접안 과정에 찍힌 AIS 데이터는 제외. 
    
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
    #         print(i)
            break
#         print(stop_index)

    # 나머지 segments의 durations 계산
    dura_list = []
    # 첫 segment 부터 
    for i in range(seg_no, len(segmenting_index_list)):
        d = pd.read_csv(f"실험/segment_test_new_25_{i}.csv")
        # 억까 index가 들어가있으면 for 문 으로 뒷부분 삭제 계산 break로 나가기 
        if d.loc[d.index[-1],"index"] > stop_index:
            for i2 in d.index:
                d_index = d.loc[i2, "index"]
                if d_index > stop_index:
                    d = d.loc[:i2,:]
                    break
        # 뒷부분이 삭제되면 오류가 나기때문에 len()이 0이 아니라면,
        if len(d) != 0:
            mean_speed = d["speed"].mean()
            damped_mean_speed = mean_speed * speed_damping_ratio
            km_h = damped_mean_speed * 1.852
            distance = reckon_point_lengths(d)
            # 걸리는 시간 list에 저장
            dura_list.append(cal_duration(km_h, distance))
    accu_timedelta = datetime.timedelta(minutes=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_timestamp + accu_timedelta
    sample.loc[sam, 'ETA'] = ETA
    # ATA 저장. ATA는 ATA = pd.to_datetime('2022-11-14 16:00:40.036669')
    sample.loc[sam, 'ATA'] = ATA
    sample.loc[sam, 'error'] = abs(ETA - ATA)

OverflowError: cannot convert float infinity to integer

In [ ]:
ddd = sample.loc[20:,:]
ddd = ddd.loc[ddd['error'].dropna(axis=0).index]
ddd

In [ ]:
ddd["eta"] = pd.to_datetime(ddd["eta"])


In [ ]:
ddd["error"] = abs(ddd["ETA"] - ddd["ATA"])

In [ ]:
ddd["AIS_ETA_error"] = abs(ddd["eta"] - ddd["ATA"])

In [ ]:
ddd.drop(columns=["mmsi", "imo", "CAPACITY",'estimated_fuel_consumption'], inplace=True)

에러가 커졌을때는 왜 커졌는지 확인 필요할듯.

if ETA > ATA:
error = ETA - ATA
else
val = abs(ETA-ATA)


In [ ]:
ddd["error"].max()

In [ ]:
ddd["error"].mean()

In [ ]:
ddd.sort_values(by="error")

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].head(100).index.values,
         bins=50, label="top 50")
plt.hist(ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].tail(100).index.values,
         bins=50, label="low 50")
plt.legend()
plt.show()

In [ ]:
ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].iloc[50]

In [ ]:
ddd.sort_values(by="error", ascending=False)[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].iloc[50]

3. 속도 계수를 조정해보자.
- 속도 계수를 적용 안했을 때
- 속도 계수를 지금과 같이 적용했을 때
- 속도 계수를 특성에 맞게 조금씩 다르게 (뒷 부분은 속도 적용하지 않는다던가 등)
- 현재 순간 속도를 보는게 아니라, 일정 시간(약 30분 ~ 60분? 실험 필요)을 두고 평균 속도나 칼만 필터를 이용한 filtered 속도를 기반으로 속도 계수 조정
    - 속도가 그렇게 차이가 없을 것 같아 별 차이는 없을지도 모름.
- 너무 속도 변화가 크거나 속도가 낮을때 제외하는 방법
- 속도가 0일때는 어떻게 계산?


# 감속률 미적용

In [47]:
# 샘플 데이터는 20부터. 
for sam in sample.index[20:stop_index]:
    re_sample = sample.loc[sam,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    current_speed = re_sample["speed"]
    current_timestamp = re_sample["timestamp"]
    current_timestamp = pd.to_datetime(current_timestamp)
    
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 999
    for i in range(len(segmenting_index_list)):
        d = pd.read_csv(f"실험/segment_test_new_25_{i}.csv")
        for idx in d.index:

            LONGITUDE = d.loc[idx, "longitude"]
            LATITUDE = d.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance < shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = d.loc[idx, "index"]
                seg_no = i
#     print(shortest_distance)
    # 마지막 인덱스 제외
    if shortest_point_idx == df_test_2.loc[df_test_2.index[-1],"index"]:
        continue
        
    # 가장 가까운 점에서 다음 점 (n+1) 선택. 
    # n+1 점과 현재 점
    df__ = df_test_2.query(f'index == {shortest_point_idx + 1}')
    new_point = (df__["latitude"].values[0],df__["longitude"].values[0])
    distance = haversine(current_point, new_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, distance)
    
    # 가장 가까운 segment 데이터 불러오기
    init_seg_df = pd.read_csv(f"실험/segment_test_new_{seg_no}.csv")
    # 마지막 세그먼트라면 제외
    if len(init_seg_df.query(f'index == {shortest_point_idx + 1}')) == 0:
        init_seg_df = pd.read_csv(f"실험/segment_test_new_{seg_no + 1}.csv")
        starting_index = init_seg_df.index[0]
    else:
        starting_index = init_seg_df.query(f'index == {shortest_point_idx + 1}').index[0]
        
    # 감속률 계산
    speed_damping_ratio = current_speed / init_seg_df["speed"].mean()
    
    # 남은 segment 간의 거리 계산 
    remained_init_seg_df = init_seg_df.loc[starting_index:,]
    remained_init_seg_df.reset_index(drop=True, inplace=True)
    accu_distance = 0
    for idx in remained_init_seg_df.index:
        if idx != 0:
            LONGITUDE = remained_init_seg_df.loc[idx, "longitude"]
            LATITUDE = remained_init_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remained_init_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remained_init_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
    # 첫번쨰 segment duration 계산
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    # 나머지 segment 계산
    # 억까 방지를 위해 접안 과정에 찍힌 AIS 데이터는 제외. 
    
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
    #         print(i)
            break
#         print(stop_index)

    # 나머지 segments의 durations 계산
    dura_list = []
    # 첫 segment 부터 
    for i in range(seg_no, len(segmenting_index_list)):
        d = pd.read_csv(f"실험/segment_test_new_25_{i}.csv")
        # 억까 index가 들어가있으면 for 문 으로 뒷부분 삭제 계산 break로 나가기 
        if d.loc[d.index[-1],"index"] > stop_index:
            for i2 in d.index:
                d_index = d.loc[i2, "index"]
                if d_index > stop_index:
                    d = d.loc[:i2,:]
                    break
        # 뒷부분이 삭제되면 오류가 나기때문에 len()이 0이 아니라면,
        if len(d) != 0:
            mean_speed = d["speed"].mean()
            damped_mean_speed = mean_speed * speed_damping_ratio
            km_h = damped_mean_speed * 1.852
            distance = reckon_point_lengths(d)
            # 걸리는 시간 list에 저장
            dura_list.append(cal_duration(km_h, distance))
    accu_timedelta = datetime.timedelta(minutes=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_timestamp + accu_timedelta
    sample.loc[sam, 'ETA'] = ETA
    # ATA 저장. ATA는 ATA = pd.to_datetime('2022-11-14 16:00:40.036669')
    sample.loc[sam, 'ATA'] = ATA
    sample.loc[sam, 'error'] = abs(ETA - ATA)

In [48]:
ddd = sample.loc[20:,:]


In [49]:
ddd = ddd.loc[ddd['error'].dropna(axis=0).index]
ddd.sort_values(by="error")

,index,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
165,389,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.5901,34.4120,17.3,174.0,2022-11-21 03:00:49.410570,2022-11-20 18:00:33,2022-11-20 02:37:09,under way using engine,301.679471,548.803883,2022-11-21 16:49:49.410570,2022-11-21 16:50:10.106911,0 days 00:00:20.696341
164,388,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.5856,34.4411,17.2,174.0,2022-11-21 02:55:46.138925,2022-11-20 17:54:28,2022-11-20 02:37:09,under way using engine,301.383856,544.356631,2022-11-21 16:49:46.138925,2022-11-21 16:50:10.106911,0 days 00:00:23.967986
248,472,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,127.8141,34.2098,16.7,70.0,2022-11-21 11:05:20.950527,2022-11-21 01:59:40,2022-11-21 01:15:06,under way using engine,124.927985,217.771271,2022-11-21 16:49:20.950527,2022-11-21 16:50:10.106911,0 days 00:00:49.156384
163,387,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.5808,34.4729,17.1,174.0,2022-11-21 02:50:42.717432,2022-11-20 17:47:47,2022-11-20 02:37:09,under way using engine,301.089980,539.956931,2022-11-21 16:48:42.717432,2022-11-21 16:50:10.106911,0 days 00:01:27.389479
292,516,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8511,34.8521,8.8,0.0,2022-11-21 16:19:52.596682,2022-11-21 06:58:54,2022-11-21 06:55:09,under way using engine,13.380367,16.833685,2022-11-21 16:51:52.596682,2022-11-21 16:50:10.106911,0 days 00:01:42.489771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,349,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.3301,35.2695,15.9,155.0,2022-11-20 23:38:55.429766,2022-11-20 14:37:04,2022-11-20 02:37:09,under way using engine,319.094918,526.150686,2022-11-22 03:24:55.429766,2022-11-21 16:50:10.106911,0 days 10:34:45.322855
124,348,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.3142,35.2946,15.8,157.0,2022-11-20 23:33:52.333842,2022-11-20 14:30:39,2022-11-20 04:13:11,under way using engine,320.786354,525.344708,2022-11-22 03:29:52.333842,2022-11-21 16:50:10.106911,0 days 10:39:42.226931
126,350,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.3446,35.2464,15.9,155.0,2022-11-20 23:43:58.819338,2022-11-20 14:42:57,2022-11-20 02:37:09,under way using engine,317.569026,523.634667,2022-11-22 03:29:58.819338,2022-11-21 16:50:10.106911,0 days 10:39:48.712427
122,346,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,125.2926,35.3217,15.7,145.0,2022-11-20 23:23:46.709602,2022-11-20 14:23:14,2022-11-20 04:13:11,under way using engine,323.036003,525.451120,2022-11-22 03:30:46.709602,2022-11-21 16:50:10.106911,0 days 10:40:36.602691


In [50]:
ddd['error'].mean()

Timedelta('0 days 02:11:48.546397432')

In [51]:
ddd['error'].max()

Timedelta('0 days 10:46:34.080454')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].head(100).index.values,
         bins=50, label="top 50")
plt.hist(ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].tail(100).index.values,
         bins=50, label="low 50")
plt.legend()
plt.show()

In [ ]:
ddd.sort_values(by="error")[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].iloc[50]

In [ ]:
ddd.sort_values(by="error", ascending=False)[[
    'index', "speed", 'timestamp', "ETA", "error", 'remained_distance'
]].iloc[50]